# 📚 LMFast: RAG-Augmented Agents

**Build knowledge-grounded AI agents with Retrieval-Augmented Generation!**

## What You'll Learn
- Create a RAG system with LMFast
- Index documents (PDF, TXT, MD)
- Query with context-aware generation
- Build a RAG-powered agent
- Save and load RAG indexes

## Why RAG for SLMs?
- Reduces hallucinations by grounding responses
- Enables domain-specific knowledge without fine-tuning
- Works great with small models (even 135M!)
- Low memory footprint

**Time to complete:** ~10 minutes

## 1️⃣ Setup

In [ ]:
# Install LMFast with RAG dependencies
!pip install -q lmfast[all] sentence-transformers faiss-cpu

import lmfast
lmfast.setup_colab_env()

import torch
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"LMFast version: {lmfast.__version__}")

## 2️⃣ Create a RAG System

LMFast's `LightweightRAG` is designed for:
- Colab T4 compatibility (low memory)
- Simple, intuitive API
- Automatic chunking and indexing

In [ ]:
from lmfast.rag import LightweightRAG, create_rag

# Sample documents about our fictional company
documents = [
    """
    TechCorp AI Solutions - Company Overview
    
    TechCorp is a leading provider of AI solutions for enterprise customers.
    Founded in 2020, we specialize in:
    - Natural Language Processing
    - Computer Vision
    - Predictive Analytics
    
    Our flagship product is SmartAssist, an AI-powered customer service platform
    that reduces support costs by 40% while improving customer satisfaction.
    """,
    """
    TechCorp Products and Pricing
    
    1. SmartAssist Basic - $99/month
       - Up to 1,000 conversations/month
       - Email support
       - Basic analytics
    
    2. SmartAssist Pro - $299/month
       - Unlimited conversations
       - 24/7 support
       - Advanced analytics
       - Custom integrations
    
    3. SmartAssist Enterprise - Custom pricing
       - On-premise deployment
       - Dedicated account manager
       - SLA guarantees
    """,
    """
    TechCorp Technical Documentation
    
    API Integration Guide:
    
    To integrate SmartAssist with your application:
    
    1. Get your API key from the dashboard
    2. Install the SDK: pip install smartassist-sdk
    3. Initialize the client:
       ```python
       from smartassist import Client
       client = Client(api_key="your-key")
       ```
    4. Send messages:
       ```python
       response = client.chat("Hello, I need help")
       ```
    
    Rate Limits:
    - Basic: 100 requests/minute
    - Pro: 1000 requests/minute
    - Enterprise: Unlimited
    """,
    """
    TechCorp Support FAQ
    
    Q: How do I reset my password?
    A: Go to Settings > Security > Reset Password
    
    Q: Can I export my conversation history?
    A: Yes, go to Analytics > Export > Select date range
    
    Q: What languages does SmartAssist support?
    A: Currently: English, Spanish, French, German, Japanese, Chinese
    
    Q: How do I upgrade my plan?
    A: Go to Billing > Change Plan > Select new plan
    """
]

print(f"Prepared {len(documents)} documents")

In [ ]:
# Create RAG system
# We'll use SmolLM for generation and MiniLM for embeddings

rag = LightweightRAG(
    model="HuggingFaceTB/SmolLM-360M-Instruct",
    embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    chunk_size=512,
    chunk_overlap=50,
    top_k=3  # Retrieve top 3 chunks
)

print("RAG system created!")

In [ ]:
# Add documents to the knowledge base
num_chunks = rag.add_documents(
    documents,
    metadata=[{"source": f"doc_{i}"} for i in range(len(documents))]
)

print(f"Indexed {num_chunks} chunks")

## 3️⃣ Query the RAG System

In [ ]:
# Simple query
questions = [
    "What is the pricing for SmartAssist Pro?",
    "How do I integrate the API?",
    "What languages are supported?",
    "What does TechCorp specialize in?"
]

print("📚 RAG Queries")
print("=" * 50)

for question in questions:
    print(f"\n❓ Question: {question}")
    
    answer = rag.query(question, max_new_tokens=150)
    print(f"💡 Answer: {answer}")
    print("-" * 40)

## 4️⃣ Query with Sources

In [ ]:
# Get answer with source chunks
question = "What are the rate limits for the API?"

answer, sources = rag.query(question, return_sources=True)

print(f"❓ Question: {question}")
print(f"\n💡 Answer: {answer}")
print(f"\n📄 Sources used ({len(sources)} chunks):")
for i, source in enumerate(sources):
    print(f"\n--- Source {i+1} ---")
    print(source[:200] + "...")

## 5️⃣ Retrieval Only

In [ ]:
# Just retrieve relevant chunks without generation
results = rag.retrieve("pricing plans", top_k=2)

print("🔍 Retrieval Results")
print("=" * 50)

for chunk, score, metadata in results:
    print(f"\nScore: {score:.3f}")
    print(f"Metadata: {metadata}")
    print(f"Content: {chunk[:200]}...")
    print("-" * 40)

## 6️⃣ Add Files to RAG

You can also add files directly (TXT, MD, PDF).

In [ ]:
# Create a sample file
sample_content = """
# TechCorp Release Notes - v2.5

## New Features
- Multi-language support for 6 new languages
- Real-time sentiment analysis
- Advanced conversation routing
- Webhook notifications

## Bug Fixes
- Fixed issue with conversation export
- Improved response latency by 30%
- Resolved authentication timeout errors

## Breaking Changes
- API v1 deprecated (sunset date: 2025-06-01)
- New authentication flow required for Enterprise
"""

with open("release_notes.md", "w") as f:
    f.write(sample_content)

# Add file to RAG
rag.add_file("release_notes.md")
print("Added release_notes.md to knowledge base")

# Query about release notes
answer = rag.query("What's new in version 2.5?")
print(f"\n❓ What's new in v2.5?")
print(f"💡 {answer}")

## 7️⃣ One-Line RAG Creation

In [ ]:
# Quick setup with create_rag
quick_rag = create_rag(
    model="HuggingFaceTB/SmolLM-135M",
    documents=[
        "Python is a programming language.",
        "JavaScript is used for web development.",
        "Rust is known for memory safety."
    ],
    top_k=2
)

answer = quick_rag.query("What is Python?")
print(f"Quick RAG answer: {answer}")

## 8️⃣ Save and Load RAG

In [ ]:
# Save RAG state
rag.save("./my_rag_index")
print("RAG saved to ./my_rag_index")

# List saved files
import os
for f in os.listdir("./my_rag_index"):
    print(f"  - {f}")

In [ ]:
# Load RAG state
loaded_rag = LightweightRAG("HuggingFaceTB/SmolLM-360M-Instruct")
loaded_rag.load("./my_rag_index")

# Verify it works
answer = loaded_rag.query("What is SmartAssist?")
print(f"Loaded RAG answer: {answer[:200]}...")

## 9️⃣ Build a RAG-Powered Agent

In [ ]:
from lmfast.agents.core import Agent, Tool
from lmfast.inference import SLMServer

# Create base model for agent
model = SLMServer("HuggingFaceTB/SmolLM-360M-Instruct")

# Create RAG as a tool
def search_knowledge_base(query: str) -> str:
    """Search the company knowledge base for information."""
    results = rag.retrieve(query, top_k=2)
    if results:
        return "\n".join([r[0] for r in results])
    return "No relevant information found."

def get_pricing(plan: str) -> str:
    """Get pricing information for a specific plan."""
    prices = {
        "basic": "$99/month",
        "pro": "$299/month",
        "enterprise": "Custom pricing - contact sales"
    }
    return prices.get(plan.lower(), "Unknown plan")

# Create agent with RAG tool
support_agent = Agent(
    model_generate_fn=model.generate,
    tools=[search_knowledge_base, get_pricing],
    system_prompt="""You are a helpful TechCorp support agent.
Use the search_knowledge_base tool to find information.
Use the get_pricing tool for pricing questions.
Always be helpful and accurate."""
)

print("RAG-powered support agent created!")

In [ ]:
# Test the RAG agent
queries = [
    "How do I integrate the SmartAssist API?",
    "What's the price of the Pro plan?",
    "What languages does your product support?"
]

print("🤖 RAG-Powered Support Agent")
print("=" * 50)

for query in queries:
    print(f"\n👤 Customer: {query}")
    response = support_agent.run(query)
    print(f"🤖 Agent: {response}")
    print("-" * 40)

## 🎉 Summary

You've learned how to:
- ✅ Create a RAG system with `LightweightRAG`
- ✅ Index documents and files
- ✅ Query with context-grounded generation
- ✅ Retrieve sources for transparency
- ✅ Save and load RAG indexes
- ✅ Build RAG-powered agents

### RAG Tips

| Tip | Why |
|-----|-----|
| Smaller chunks (256-512) | Better retrieval precision |
| Overlap (50-100 chars) | Avoid cutting context |
| Higher top_k for complex Q | More context helps |
| Domain-specific embeddings | Better matching |

### Next Steps
- `13_guardrails.ipynb` - Add safety to your RAG
- `15_browser_deployment.ipynb` - Deploy RAG in browser